Bu kodun hazırlanmasında Unsloth dokümantasyonundan yararlanmıştır.

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.7/346.7 kB 9.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 96.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.4/270.4 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.9 MB/s eta 0:00:00:00:0100:

Büyük Dil Modellerini olduğu gibi eğitmek için uzun zaman, fazla kaynak ve büyük veriye ihtiyaç vardır. Bu sorunun önüne geçmek için bir takım kütüphaneler ve yöntemler kullanılır. Kullanılan kütüphanelerden birisi de unsloth kütüphanesidir. Bu kütüphane istenilen açık kaynaklı büyük dil modellerini sisteme çeker ve çeşitli yöntemler sayesinde yüksek GB gereken veriyi küçülterek eğitimin kolay gerçekleştirilmesini ve çıktıların istenilen yerlerde saklanmasını sağlar. Unsloth kütüphanesinden kısaca bahsettiğimize göre yöntemlere değinelim.

Bahsedilen ve kodta geçen yöntemlerden birisi de 4bit kuantizasyon tekniğidir. Model ağırlıkları genellikle 32 bit float ya da 16 bit float olarak saklanır. Büyük modellerde bu ağırlıklar sistemde çok fazla yer kaplar. 4 bit kuantizasyon yöntemi ile ağırlıklar normalden 4 ile 8 kat daha az yer kaplar. Ancak saklanan verinin kesinliğinin (precision) düşük 16 ve 32 bitliğe göre daha düşük olmasından dolayı sonuçlar teorik olarak biraz düşük çıkabilir. Ancak modern tekniklerle (örn. Quantization-Aware Training, GPTQ) kayıp minimuma indirilebilir.

## Quantization-Aware Training (QAT)

Modeli FP32 veya FP16 olarak eğitirken aynı zamanda ağırlıkların kuantize edilmiş hallerini de simüle eder. Eğitim sırasında ağırlıklar hem float hem de kuantize edilmiş halde bulunur. Bu yöntemin amacı kuantizasyon sonrası performans kaybını önceden tahmin etmek ve minimize etmektir. Forward pass sırasında model, kuantize edilmiş ağırlıkları kullanır. Backpropagation sırasında gradient’ler float ağırlıklar üzerinden hesaplanır. Böylece model, kuantize edilmiş ağırlıkların getirdiği hataya "alışır" ve performans kaybı minimize edilir. Eğitim süresi artar, çünkü kuantizasyon simülasyonu ek hesaplama gerektirir.


## GPTQ (Gradient-based Post-Training Quantization)

Büyük modelleri eğittikten sonra kuantize eder. Gradient kullanarak 4-bit veya 3-bit kuantizasyon sırasında hatayı minimize eder. Model eğitildikten sonra ağırlıklar alınır. GPTQ, ağırlıkları grup veya blok bazlı olarak kuantize eder.
Kuantizasyon hatasını minimize etmek için local gradient’ler kullanılır (gradient descent ile optimize edilir, ama model yeniden eğitilmez). Sonuç olarak 4-bit veya 3-bit ağırlıklar, orijinal model performansına çok yakın bir doğrulukla elde edilir. Modeli yeniden eğitmek gerekmez. Post-training adımı, özellikle büyük modellerde zaman alabilir.

In [2]:
from unsloth import FastModel
import torch

max_seq_length = 1024
# Seçeneklere full_finetuning parametresi de eklenmiş ama model boyutunu da ele aldığımızda çok da gerek yok gibi
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, 
    load_in_4bit = False)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-24 20:57:11.783280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761339432.153700      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761339432.271571      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.9: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [3]:
model = FastModel.get_peft_model(
    model, 
    r = 8, 
    lora_alpha = 8,
    lora_dropout = 0,
    bias = 'none'
)

Unsloth: Making `model.base_model.model.model` require gradients


In [36]:
from datasets import load_dataset
dataset = load_dataset("openai/gsm8k", "main", split = "train")
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [37]:
# Veri setinden örnek bir kısım alalım
dataset[0]['question'], dataset[0]['answer']

('Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72')

In [38]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [42]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [43]:
dataset[0]['question'], dataset[0]['answer']

('Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72')

In [60]:
import re

def split_question_answer(example):
    text = example["text"].strip()
    
    # <<...>> içindeki işlem açıklamalarını temizle ve sonucu al
    text = re.sub(r"<<.*?=([0-9\.]+)>>(\1)?", r"\1", text)
    
    # question ve reasoning kısmını ayır (#### işareti ile)
    if "####" in text:
        reasoning, answer = text.rsplit("####", 1)
        answer = answer.strip()
    else:
        reasoning = text
        answer = ""
    
    # question = ilk satır
    lines = reasoning.split("\n")
    question = lines[0].strip()
    reasoning_text = "\n".join(lines[1:]).strip()
    
    return {
        "question": question,
        "reasoning": reasoning_text,
        "answer": answer
    }

dataset3 = dataset2.map(split_question_answer)


In [63]:
dataset3[0]['reasoning']

'Natalia sold 48/2 = 24 clips in May. Natalia sold 48+24 = 72 clips altogether in April and May.'

In [64]:
def prepare_for_sft(example):
    prompt = example["question"]
    target = example["reasoning"] + "\nFinal Answer: " + example["answer"]
    text = f"Question: {prompt} \nAnswer: {target}"
    return {"text": text}

dataset_sft = dataset3.map(prepare_for_sft)


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [65]:
dataset = dataset_sft

Modeli eğitmeden önce bütün modeli eğitmek fazlaca yer ve zaman içereceğinden dolayı PEFT (Parameter Efficient Fine Tuning) tekniğini kullanırız. Bu bize belli katmanlar üzerinde model eğitimini sağlar ve bütün modeli eğitmektense modelin başlı kısımlarını eğitiriz. Ayrıca PEFT ile günümüzde sıkça kullanılan ve PEFT'in alt yöntemlerinden biri olan LORA (Low Rank Adaptation) tekniğini de sıkça kullanırız.

### PEFT’in alt yöntemleri:

 PEFT bir çatı kavramdır. Altında farklı teknikler vardır:

###
Teknik	        Açıklama

LoRA:	        En popüler yöntem. Ağırlıklara düşük-rank matris ekler.

Prefix Tuning:	Girdi dizisine “özel tokenlar” ekleyerek ayar yapar.

Adapter Tuning:	Her katmana küçük ek sinir ağı modülleri ekler.

Prompt Tuning:	Modeli yeniden eğitmeden, giriş metnine sanal promptlar ekler.
###

Bugün en çok kullanılanı LoRA’dır.

### LoRA (Low-Rank Adaptation)

LoRA, büyük matrisleri tamamen güncellemek yerine, onlara küçük, düşük-rank eklemeler yapar. Bir model katmanında normalde şöyle bir ağırlık vardır:

𝑦 = 𝑊 * 𝑥

Burada 𝑊 genelde çok büyük (örneğin 4096×4096 boyutunda) bir matristir. LoRA bunu şu şekilde değiştirir:

𝑦 =(𝑊+𝐴𝐵)𝑥

Burada 𝐴 ve 𝐵 küçük, düşük-rank matrislerdir. 𝑟 bu matrislerin boyutunu belirleyen parametre (örneğin 4, 8, 16). Eğitim sırasında 𝑊 sabit kalır (freeze). Sadece 𝐴 ve 𝐵 öğrenilir. Bu sayede parametre sayısı 100 kat azalabilir. Performans çoğu zaman aynı kalır veya çok az düşer. Bu sayede eğitilecek parametre sayısı azalır, bellekte tasarruf sağlanır ve eğitim hızlanır.

![image.png](attachment:c07ed10e-db72-4371-9bef-f25967c697c6.png)

### PEFT + LoRA birlikte nasıl çalışır?

PEFT, “hangi parametreleri eğitelim?” sorusuna genel cevaptır. LoRA, bu stratejilerden biridir (en verimlisi). PEFT önceden eğitilmiş katmanları alır ve belirli modüllere LoRA ekler. LoRA ise bu katmanları eğitir. 

In [66]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/7473 [00:00<?, ? examples/s]

In [67]:
trainer_stats = trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 6,522,880 of 1,006,408,832 (0.65% trained)


Step,Training Loss
1,1.851400
2,1.868300
3,1.924200
4,1.870600
5,1.713000
6,1.647800
7,1.653900
8,1.478200
9,1.589700
10,1.566400


In [72]:
import torch

# Soruyu hazırla
prompt = """Emily had 150 apples. She gave 2/5 of them to her friends and then sold half of the remaining apples at the market. How many apples does Emily have left?"""

# Tokenize et
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate kullan
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.5,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
    )

# Decode
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)


Emily had 150 apples. She gave 2/5 of them to her friends and then sold half of the remaining apples at the market. How many apples does Emily have left? 
Answer: Emily gave 2/5 * 150 = 60 apples to her friends. She had 150 - 60 = 90 apples left. She sold half of the remaining 90 apples at the market, so she sold 90 / 2 = 45 apples. She has 90 - 45 = 45 apples left.
Final Answer: 45
Final Answer: 45


In [75]:
model.save_pretrained("gemma3_model")

In [76]:
tokenizer.save_pretrained("gemma3_tokenizer")

('gemma3_tokenizer/tokenizer_config.json',
 'gemma3_tokenizer/special_tokens_map.json',
 'gemma3_tokenizer/chat_template.jinja',
 'gemma3_tokenizer/tokenizer.model',
 'gemma3_tokenizer/added_tokens.json',
 'gemma3_tokenizer/tokenizer.json')

In [77]:
import shutil

# Zip adı ve klasörler
shutil.make_archive("gemma3_model_and_tokenizer", 'zip', root_dir="gemma3_model")
shutil.make_archive("gemma3_tokenizer", 'zip', root_dir="gemma3_tokenizer")


'/kaggle/working/gemma3_tokenizer.zip'